# Schedule Access Plan Recommendation

This notebook is dedicated to model fitting in terms of database access plans.

In [1]:
# pandas
import pandas as pd
print('pandas: %s' % pd.__version__)
# numpy
import numpy as np
print('numpy: %s' % np.__version__)
# sklearn
import sklearn as sk
from sklearn import preprocessing
print('sklearn: %s' % sk.__version__)

pandas: 0.23.4
numpy: 1.15.2
sklearn: 0.19.0


In [2]:
#
# Experiment Config
tpcds='TPCDS1' # Schema upon which to operate test
y_labels = ['COST',
            'CARDINALITY',
            'BYTES',
            'CPU_COST',
            'IO_COST',
            'TEMP_SPACE',
            'TIME']

### Read data from file into pandas dataframes

In [3]:
rep_vsql_plan_path = 'C:/Users/gabriel.sammut/University/Data_ICS5200/Schedule/' + tpcds + '/v2/rep_vsql_plan.csv'
#rep_vsql_plan_path = 'D:/Projects/Datagenerated_ICS5200/Schedule/' + tpcds + '/v2/rep_vsql_plan.csv'
#
rep_vsql_plan_df = pd.read_csv(rep_vsql_plan_path,dtype=str)
print(rep_vsql_plan_df.head())
#
def prettify_header(headers):
    """
    Cleans header list from unwated character strings
    """
    header_list = []
    [header_list.append(header.replace("(","").replace(")","").replace("'","").replace(",","")) for header in headers]
    return header_list
#
rep_vsql_plan_df.columns = prettify_header(rep_vsql_plan_df.columns.values)
print('------------------------------------------')
print(rep_vsql_plan_df.columns)

     ('SQL_ID',) ('PLAN_HASH_VALUE',) ('ID',)    ('OPERATION',) ('OPTIONS',)  \
0  dxv968j0352kb            103598129       0  SELECT STATEMENT          NaN   
1  dxv968j0352kb            103598129       1              SORT     GROUP BY   
2  dxv968j0352kb            103598129       2    PX COORDINATOR          NaN   
3  dxv968j0352kb            103598129       3           PX SEND  QC (RANDOM)   
4  dxv968j0352kb            103598129       4              SORT     GROUP BY   

  ('OBJECT_NODE',) ('OBJECT_OWNER',) ('OBJECT_NAME',) ('OBJECT_ALIAS',)  \
0              NaN               NaN              NaN               NaN   
1              NaN               NaN              NaN               NaN   
2              NaN               NaN              NaN               NaN   
3           :Q1001               SYS         :TQ10001               NaN   
4           :Q1001               NaN              NaN               NaN   

  ('OBJECT_TYPE',)         ...          ('SEARCH_COLUMNS',) ('COST',

### Dealing with empty values

In [4]:
def get_na_columns(df, headers):
    """
    Return columns which consist of NAN values
    """
    na_list = []
    for head in headers:
        if df[head].isnull().values.any():
            na_list.append(head)
    return na_list
#
print('N/A Columns\n')
print('\nREP_VSQL_PLAN Features ' + str(len(rep_vsql_plan_df.columns)) + ': ' + str(get_na_columns(df=rep_vsql_plan_df,headers=rep_vsql_plan_df.columns)) + "\n")
#
def fill_na(df):
    """
    Replaces NA columns with 0s
    """
    return df.fillna(0)
#
# Populating NaN values with amount '0'
df = fill_na(df=rep_vsql_plan_df)

N/A Columns


REP_VSQL_PLAN Features 22: ['OPTIONS', 'OBJECT_NODE', 'OBJECT_OWNER', 'OBJECT_NAME', 'OBJECT_ALIAS', 'OBJECT_TYPE', 'OPTIMIZER', 'COST', 'CARDINALITY', 'BYTES', 'CPU_COST', 'IO_COST', 'TEMP_SPACE', 'TIME', 'QBLOCK_NAME']



### Feature Selection

In this step, redundant features are dropped. Features are considered redundant if exhibit a standard devaition of 0 (meaning no change in value).

In [5]:
def drop_flatline_columns(df):
    columns = df.columns
    flatline_features = []
    for i in range(len(columns)):
        try:
            std = df[columns[i]].std()
            if std == 0:
                flatline_features.append(columns[i])
        except:
            pass
    #
    #print('Features which are considered flatline:\n')
    #for col in flatline_features:
    #    print(col)
    print('\nShape before changes: [' + str(df.shape) + ']')
    df = df.drop(columns=flatline_features)
    print('Shape after changes: [' + str(df.shape) + ']')
    print('Dropped a total [' + str(len(flatline_features)) + ']')
    return df
#
df = drop_flatline_columns(df=df)
print('\nAfter flatline column drop:')
print(df.shape)
print(df.columns)
print(df.head())


Shape before changes: [(98794, 22)]
Shape after changes: [(98794, 22)]
Dropped a total [0]

After flatline column drop:
(98794, 22)
Index(['SQL_ID', 'PLAN_HASH_VALUE', 'ID', 'OPERATION', 'OPTIONS',
       'OBJECT_NODE', 'OBJECT_OWNER', 'OBJECT_NAME', 'OBJECT_ALIAS',
       'OBJECT_TYPE', 'OPTIMIZER', 'DEPTH', 'SEARCH_COLUMNS', 'COST',
       'CARDINALITY', 'BYTES', 'CPU_COST', 'IO_COST', 'TEMP_SPACE', 'TIME',
       'QBLOCK_NAME', 'TIMESTAMP'],
      dtype='object')
          SQL_ID PLAN_HASH_VALUE ID         OPERATION      OPTIONS  \
0  dxv968j0352kb       103598129  0  SELECT STATEMENT            0   
1  dxv968j0352kb       103598129  1              SORT     GROUP BY   
2  dxv968j0352kb       103598129  2    PX COORDINATOR            0   
3  dxv968j0352kb       103598129  3           PX SEND  QC (RANDOM)   
4  dxv968j0352kb       103598129  4              SORT     GROUP BY   

  OBJECT_NODE OBJECT_OWNER OBJECT_NAME OBJECT_ALIAS OBJECT_TYPE  \
0           0            0           0  

### Floating point precision conversion

Each column is converted into a column of type values which are floating point for higher precision.

In [6]:
print(df.shape)
df[y_labels] = df[y_labels].astype(float)
df[y_labels] = np.round(df[y_labels], 3) # rounds to 3 dp
print(df.shape)

(98794, 22)
(98794, 22)


### Label Encoding

Converting labels/features into numerical representations

In [7]:
def encode(df, encoded_labels):
    for col in df.columns:
        if col in encoded_labels:
            le = preprocessing.LabelEncoder()
            df[col] = le.fit_transform(df[col].astype(str))
    return df
#
# Determine labels used for encoding
encoded_labels = ['OPERATION','OPTIONS','OBJECT_OWNER','OBJECT_NAME','OBJECT_ALIAS','OBJECT_TYPE','OPTIMIZER','QBLOCK_NAME']
df = encode(df=df, encoded_labels=encoded_labels)
print('Encoded labels:\n' + str(encoded_labels) + "\n\n----------------------------------------------\n\n")
print(df.head())

Encoded labels:
['OPERATION', 'OPTIONS', 'OBJECT_OWNER', 'OBJECT_NAME', 'OBJECT_ALIAS', 'OBJECT_TYPE', 'OPTIMIZER', 'QBLOCK_NAME']

----------------------------------------------


          SQL_ID PLAN_HASH_VALUE ID  OPERATION  OPTIONS OBJECT_NODE  \
0  dxv968j0352kb       103598129  0         28        0           0   
1  dxv968j0352kb       103598129  1         30       14           0   
2  dxv968j0352kb       103598129  2         24        0           0   
3  dxv968j0352kb       103598129  3         26       26      :Q1001   
4  dxv968j0352kb       103598129  4         30       14      :Q1001   

   OBJECT_OWNER  OBJECT_NAME  OBJECT_ALIAS  OBJECT_TYPE         ...           \
0             0            0             0            0         ...            
1             0            0             0            0         ...            
2             0            0             0            0         ...            
3             1           11             0            0         ...     

### Data Ordering

Sorting of datasets in order of 
* TIMESTAMP
* PLAN_HASH_VALUE
* ID

In [8]:
df.sort_values(by=['TIMESTAMP','PLAN_HASH_VALUE','ID'], ascending=True, inplace=True)
print(df.head())

              SQL_ID PLAN_HASH_VALUE ID  OPERATION  OPTIONS OBJECT_NODE  \
0      dxv968j0352kb       103598129  0         28        0           0   
50644  dxv968j0352kb       103598129  0         28        0           0   
1      dxv968j0352kb       103598129  1         30       14           0   
50645  dxv968j0352kb       103598129  1         30       14           0   
2      dxv968j0352kb       103598129  2         24        0           0   

       OBJECT_OWNER  OBJECT_NAME  OBJECT_ALIAS  OBJECT_TYPE  \
0                 0            0             0            0   
50644             0            0             0            0   
1                 0            0             0            0   
50645             0            0             0            0   
2                 0            0             0            0   

              ...           SEARCH_COLUMNS   COST CARDINALITY  BYTES  \
0             ...                        0  880.0         0.0    0.0   
50644         ...         

### Access Plan Resource Aggregation

This method attempts to tackle the problem of access plan anomolies by aggregating resources per explain plan. Notable resources which are being considered are as follows:

* COST
* CARDINALITY
* BYTES
* PARTITION_DELTA (Partition End - Partition Start)
* CPU_COST
* IO_COST
* TEMP_SPACE
* TIME

The reasoning behind these fields in particular is mainly because these columns can be aggregated together.

In [9]:
print(df_aggregate.shape)
df_aggregate = df.groupby(['SQL_ID','PLAN_HASH_VALUE']).sum()
df_aggregate.reset_index(inplace=True)
print(df_aggregate.columns)
print(df_aggregate.shape)

Index(['SQL_ID', 'PLAN_HASH_VALUE', 'OPERATION', 'OPTIONS', 'OBJECT_OWNER',
       'OBJECT_NAME', 'OBJECT_ALIAS', 'OBJECT_TYPE', 'OPTIMIZER', 'COST',
       'CARDINALITY', 'BYTES', 'CPU_COST', 'IO_COST', 'TEMP_SPACE', 'TIME',
       'QBLOCK_NAME'],
      dtype='object')
(270, 17)
